In [1]:
import pandas as pd

In [2]:
path = '/opt/ml/input/data'

train = pd.read_csv(path + '/train_data.csv')
test = pd.read_csv(path + '/test_data.csv')

In [3]:
from recommenders.utils import gpu_utils

print("GPUs:\n", gpu_utils.get_gpu_info())

GPUs:
 [{'device_name': 'Tesla V100-PCIE-32GB', 'total_memory': 32510.5, 'free_memory': 32201.5}]


In [4]:
# train, test 병합. 모든 데이터 활용하기.
dat = pd.concat([train, test], axis = 0)
dat = dat.sort_values(by = ['userID', 'Timestamp'])

In [5]:
data = dat[['userID', 'assessmentItemID', 'answerCode']].copy().reset_index(drop=True)
data.columns = ['userID', 'itemID', 'rating']

In [6]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

In [7]:
data.itemID = encoder.fit_transform(data.itemID)

In [8]:
data

,userID,itemID,rating
0,0,5354,1
1,0,5355,1
2,0,5356,1
3,0,5357,1
4,0,5358,1
...,...,...,...
2526695,7441,2373,0
2526696,7441,3909,1
2526697,7441,3910,1
2526698,7441,3911,1


In [9]:
# rating이 -1로 되어있는 문제를 맞추기 위해 사용.
_train = data[data['rating'] >= 0]
_test = data[data['rating'] < 0]

In [10]:
# set the environment path to find Recommenders
import sys
import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.spark_splitters import spark_random_split
from recommenders.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation
from recommenders.utils.spark_utils import start_or_get_spark

print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))


System version: 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
Spark version: 3.3.1


In [11]:
# the following settings work well for debugging locally on VM - change when running on a cluster
# set up a giant single executor with many threads and specify memory cap

spark = start_or_get_spark("ALS PySpark", memory="16g")
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/06 06:56:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [24]:
from pyspark.sql import SQLContext
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

sparkdf_train = sqlContext.createDataFrame(_train)
sparkdf_test = sqlContext.createDataFrame(_test)

TypeError: createDataFrame() got an unexpected keyword argument 'size'

In [13]:
# Column names for the dataset
COL_USER = "userID"
COL_ITEM = "itemID"
COL_RATING = "rating"

In [14]:
sparkdf_train.show()

22/12/06 06:58:06 WARN TaskSetManager: Stage 0 contains a task of very large size (2876 KiB). The maximum recommended task size is 1000 KiB.


+------+------+------+
|userID|itemID|rating|
+------+------+------+
|     0|  5354|     1|
|     0|  5355|     1|
|     0|  5356|     1|
|     0|  5357|     1|
|     0|  5358|     1|
|     0|  5359|     1|
|     0|  5367|     0|
|     0|  5368|     1|
|     0|  5369|     1|
|     0|  5370|     1|
|     0|  5371|     1|
|     0|  5372|     1|
|     0|  5373|     1|
|     0|  5381|     1|
|     0|  5382|     1|
|     0|  5383|     1|
|     0|  5384|     1|
|     0|  5385|     1|
|     0|  5386|     0|
|     0|  5387|     1|
+------+------+------+
only showing top 20 rows



In [26]:
sparkdf_test.show()

+------+------+------+
|userID|itemID|rating|
+------+------+------+
|     3|  4965|    -1|
|     4|  7748|    -1|
|    13|  7484|    -1|
|    17|  9381|    -1|
|    26|  6231|    -1|
|    29|  1967|    -1|
|    45|  3759|    -1|
|    53|  3779|    -1|
|    58|  7847|    -1|
|    64|  7748|    -1|
|    76|  5071|    -1|
|    79|  7888|    -1|
|    94|   746|    -1|
|   120|  5099|    -1|
|   159|  5022|    -1|
|   172|  1675|    -1|
|   193|  7788|    -1|
|   200|  1736|    -1|
|   203|  5820|    -1|
|   204|  7740|    -1|
+------+------+------+
only showing top 20 rows



In [28]:
header = {
    "userCol": COL_USER,
    "itemCol": COL_ITEM,
    "ratingCol": COL_RATING
}


als = ALS(
    rank=10,
    maxIter=15,
    implicitPrefs=False,
    regParam=0.05,
    coldStartStrategy='drop',
    nonnegative=False,
    seed=42,
    **header
)

In [16]:
model = als.fit(sparkdf_train)

22/12/06 06:58:08 WARN TaskSetManager: Stage 1 contains a task of very large size (2876 KiB). The maximum recommended task size is 1000 KiB.
22/12/06 06:58:08 WARN TaskSetManager: Stage 2 contains a task of very large size (2876 KiB). The maximum recommended task size is 1000 KiB.


22/12/06 06:58:14 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/12/06 06:58:14 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/12/06 06:58:14 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [27]:
model

ALSModel: uid=ALS_675a38bcd6b5, rank=10

In [18]:
# Get the cross join of all user-item pairs and score them.
users = sparkdf_train.select(COL_USER).distinct()
items = sparkdf_train.select(COL_ITEM).distinct()
user_item = users.crossJoin(items)
dfs_pred = model.transform(user_item)


In [21]:
dfs_pred_exclude_sparkdf_train = dfs_pred.alias("pred").join(
        sparkdf_train.alias("sparkdf_train"),
        (dfs_pred[COL_USER] == sparkdf_train[COL_USER]) & (dfs_pred[COL_ITEM] == sparkdf_train[COL_ITEM]),
        how='outer'
    )

top_all = dfs_pred_exclude_sparkdf_train.filter(dfs_pred_exclude_sparkdf_train[f"sparkdf_train.{COL_RATING}"].isNull()) \
    .select('pred.' + COL_USER, 'pred.' + COL_ITEM, 'pred.' + "prediction")

22/12/06 07:01:15 WARN Column: Constructing trivially true equals predicate, 'userID#0L = userID#0L'. Perhaps you need to use aliases.


In [37]:
top_all.show()

+------+------+----------+
|userID|itemID|prediction|
+------+------+----------+
|     0|   181|0.53539914|
|     0|   189|  0.853383|
|     0|   428| 0.6558904|
|     0|   444|0.84482723|
|     0|   624| 0.8378153|
|     0|  1054|0.86947024|
|     0|  1319| 0.5791196|
|     0|  1937|0.45302948|
|     0|  2042|0.33870345|
|     0|  2066| 0.5402655|
|     0|  2109| 0.7932534|
|     0|  2307| 0.6252688|
|     0|  2417| 0.7564861|
|     0|  2466|0.81148344|
|     0|  2618|0.29677218|
|     0|  2660|0.20296177|
|     0|  3484| 0.3577386|
|     0|  3492| 0.6349364|
|     0|  3698|0.77546144|
|     0|  3952| 0.5189643|
+------+------+----------+
only showing top 20 rows



In [38]:
rank_eval = SparkRankingEvaluation(sparkdf_test, top_all, k = 10, col_user=COL_USER, col_item=COL_ITEM, 
                                    col_rating=COL_RATING, col_prediction="prediction", 
                                    relevancy_method="top_k")

In [39]:
print("Model:\tALS",
      "Top K:\t%d" % rank_eval.k,
      "MAP:\t%f" % rank_eval.map_at_k(),
      "NDCG:\t%f" % rank_eval.ndcg_at_k(),
      "Precision@K:\t%f" % rank_eval.precision_at_k(),
      "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n')

Model:	ALS
Top K:	10
MAP:	0.000224
NDCG:	0.000479
Precision@K:	0.000134
Recall@K:	0.001344


In [41]:
prediction = model.transform(sparkdf_test)
prediction.cache().show()

+------+------+------+----------+
|userID|itemID|rating|prediction|
+------+------+------+----------+
|  2769|  3794|    -1|0.41963077|
|   792|  6336|    -1|0.69257253|
|  1469|  7880|    -1| 0.2824658|
|  7058|  7880|    -1|0.23967999|
|  3754|  7880|    -1|0.21793966|
|  4870|  7880|    -1| 0.3223104|
|  4969|  7880|    -1|0.26041043|
|  3872|  7880|    -1|0.21172176|
|  6830|  7880|    -1|0.30843943|
|  3533|  7880|    -1|0.14722449|
|  1390|  7880|    -1|0.20747274|
|   616|  7880|    -1|0.21332514|
|  2282|  7880|    -1|0.35188657|
|  1044|  7880|    -1| 0.2682323|
|   617|  7880|    -1|0.35975605|
|   694|  9427|    -1| 0.4238108|
|  2577|  1721|    -1|  0.521191|
|   656|  1721|    -1|0.10192484|
|  4450|  4929|    -1|0.20404087|
|    76|  5071|    -1| 0.4873258|
+------+------+------+----------+
only showing top 20 rows



In [58]:
prediction = prediction.toPandas()

In [60]:
prediction = prediction['prediction'].tolist()

In [61]:
prediction

[0.419630765914917,
 0.6925725340843201,
 0.28246578574180603,
 0.2396799921989441,
 0.21793965995311737,
 0.3223103880882263,
 0.2604104280471802,
 0.2117217630147934,
 0.3084394335746765,
 0.14722448587417603,
 0.20747274160385132,
 0.2133251428604126,
 0.3518865704536438,
 0.2682322859764099,
 0.3597560524940491,
 0.42381080985069275,
 0.5211910009384155,
 0.1019248366355896,
 0.20404087007045746,
 0.4873257875442505,
 0.6845792531967163,
 0.6920206546783447,
 0.6860316395759583,
 0.2599851191043854,
 0.6079626679420471,
 0.3418430984020233,
 0.4275074005126953,
 0.6095579266548157,
 0.8102130889892578,
 0.007544219493865967,
 0.6801564693450928,
 0.5760509967803955,
 0.5817609429359436,
 0.7231013178825378,
 0.3260524868965149,
 0.6609612703323364,
 0.6806880235671997,
 0.5330232381820679,
 0.2214895337820053,
 0.6620646119117737,
 0.20987215638160706,
 0.3318711519241333,
 0.3075534701347351,
 0.6114686727523804,
 0.4472002685070038,
 0.4956579804420471,
 0.011042817495763302,
 0.

In [62]:
submission = pd.DataFrame(columns=['id','prediction'])

In [63]:
submission['id'] = list(i for i in range(744))
submission['prediction'] = prediction

In [64]:
submission.to_csv('./output/recommenders_ALS.csv', index = False)